In [37]:
from datetime import datetime
import sys
import time
import requests
import html


api_url = "https://climatekg.semanticclimate.net/api.php"

def fetch_mediawiki_data(api_url, params):
    try:
        response = requests.get(api_url, params=params)
        response.raise_for_status()  # Raise an exception for HTTP errors
        data = response.json()
        return data
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from MediaWiki API: {e}")
        return None

def extract_items(data):
    items = []
    results = data.get("query", {}).get("results", {})
    
    for item_id, item_data in results.items():
        #print(item_data)
        
        multilang_fields = ["description"]
        entity_fields = ["subCategoryOf", "synonyms", "similarTo"]
        txt_fields = ["term","description_plain","vocabulary","acronym","tags"]
        item = {
            "id": item_id
        }

        for fieldname in txt_fields:
            if item_data["printouts"].get(fieldname, [{}]):
                #print(fieldname)
                item[fieldname] = item_data["printouts"].get(fieldname, [None])[0]
            else:
                item[fieldname] = ""
        
        for fieldname in entity_fields:
            if item_data["printouts"].get(fieldname, [{}]):
                #print(fieldname)
                item[fieldname] = str(item_data["printouts"].get(fieldname, [None]))
            else:
                item[fieldname] = ""

        for fieldname in multilang_fields:
            if item_data["printouts"].get(fieldname, [{}]):
                #print(fieldname)
                item[fieldname] = item_data["printouts"].get(fieldname, [{}])[0].get("Text", {}).get("item", [None])[0]
            else:
                item[fieldname] = ""
        

        items.append(item)
    
    return items

def generate_html_table(items):
    html = '<table border="1">'
    # Header
    html += '<tr>'
    html += '<th>ID</th>'
    html += '<th>Term</th>'
    html += '<th>Definition</th>'
    html += '<th>SubCategory Of</th>'
    html += '</tr>'
    
    # Rows
    for item in items:
        html += '<tr>'
        html += f'<td>{item["id"]}</td>'
        html += f'<td>{item["term"]}</td>'
        html += f'<td>{item["definition"]}</td>'
        html += f'<td>{item["subCategoryOf"]}</td>'
        html += '</tr>'
    
    html += '</table>'
    return html

def generate_html_span(items, vocabulary):
    
    html = f'## <span>{vocabulary}</span>'      
    for item in items:  
        html += f'''
        
            ### <span class="gloss-term-all gloss-term">{item["term"]}</span> 

            <span class="gloss-term-all gloss-definition">{item["description"]}</span>   

            **<span class="gloss-term-all gloss-cl-definition-label">{item["description_plain"]}</span>**
      
            <span class="gloss-term-all gloss-cl-definition">Klartextbeschreibung (label)</span>
      
            **<span class="gloss-term-all gloss-acronym-label">{item["acronym"]}</span>**   
      
            <span class="gloss-term-all gloss-acronym">Akronym (label)</span>
      
            **<span class="gloss-term-all gloss-tag-label">{item["tags"]}</span>**
      
            <span class="gloss-term-all gloss-tag">Tag (label)</span>
      
            **<span class="gloss-term-all gloss-similar-tag gloss-similar-label">{item["similarTo"]}</span>**
      
            <span class="gloss-term-all gloss-similar-tag">Ähnlich (label)</span>
      
            **<span class="gloss-term-all gloss-subclass-tag gloss-subclass-label">{item["subCategoryOf"]}</span>**
      
            <span class="gloss-term-all gloss-subclass-tag">Unterklasse von (label)</span>
      
            **<span class="gloss-term-all gloss-synonyme-tag gloss-synonyme-label">{item["synonyms"]}</span>**
      
            <span class="gloss-term-all gloss-synonyme-tag">Synonyme (label)</span>'''
    return html

def get_glossary(vocabulary):

    params = {
        'action': 'ask',
        'query': f'[[Item:+]] [[has subobject::+]] [[P1::Term]] [[P18::{vocabulary}]]|?Wikibase description=description |?P26=description_plain |?P3=term |?P10.P3=subCategoryOf |?P21=acronym |?P22=tags |?P23=similarTo |?P18=vocabulary |?P24=synonyms',
        'format': 'json'
    }

    response = fetch_mediawiki_data(api_url, params)
    items = extract_items(response)
    
    if items:
        # Generate the HTML for the table
        html = '<html><head><link rel="stylesheet" href="glossary.css"></head><body>'
        #html += generate_html_table(items)
        html += generate_html_span(items,vocabulary)
        html += '</body></html>'
        print(html)





In [39]:
get_glossary("CoSite")

<html><head><link rel="stylesheet" href="glossary.css"></head><body>## <span>CoSite</span>
        
            ### <span class="gloss-term-all gloss-term">Co-Site</span> 

            <span class="gloss-term-all gloss-definition">Co-Kreation in der Region - Systematisch und innovativ Transfer entwickeln</span>   

            **<span class="gloss-term-all gloss-cl-definition-label"></span>**
      
            <span class="gloss-term-all gloss-cl-definition">Klartextbeschreibung (label)</span>
      
            **<span class="gloss-term-all gloss-acronym-label"></span>**   
      
            <span class="gloss-term-all gloss-acronym">Akronym (label)</span>
      
            **<span class="gloss-term-all gloss-tag-label">Forschungsprojekt</span>**
      
            <span class="gloss-term-all gloss-tag">Tag (label)</span>
      
            **<span class="gloss-term-all gloss-similar-tag gloss-similar-label"></span>**
      
            <span class="gloss-term-all gloss-similar-tag